## Prepping features dataset


Things to look at:
1. We are currently dropping NA in Demand, Temp. and Humidity. --> Temp and humidity is given each 5 minutes, fill up temp. and humidity with average. 
2. Normalization

In [14]:
import math, datetime, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]

In [2]:
def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df

In [15]:
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
train

,Unnamed: 0,datetime_local,demand_kW
0,0,12/31/18 21:15,2064.101392
1,1,12/31/18 21:30,1874.002081
2,2,12/31/18 21:45,1988.168511
3,3,12/31/18 22:00,2022.795943
4,4,12/31/18 22:15,1986.981872
5,5,12/31/18 22:30,1921.142106
6,6,12/31/18 22:45,1956.130266
7,7,12/31/18 23:00,2039.546964
8,8,12/31/18 23:15,2100.701840
9,9,12/31/18 23:30,1901.260583


In [16]:
#Create new dummy dfs
base_df = train.copy()
base_weather = weather.copy()

#Remove unnecessary columns
base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})
#base_weather['datetime_local'] = [datetime.datetime.strptime(x, 'yyyy/MM/dd HH:mm:SS') for x in base_weather['localstrptime']]

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame 
base_df = pd.concat([base_df, base_weather], axis=1)
base_df = pd.concat

#Concatenate the weight features



# Drop all NaN values\n"
base_df.dropna(subset=['demand_kW', 'Temperature'])


,demand_kW,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,hour
datetime_local,,,,,,,,,,,,
2018-12-31 21:15:00,2064.101392,2018.0,12.0,0.0,31.0,21.0,15.0,2018-12-31 21:15:00-06:00,61.27,46.40,2019-01-01 03:15:00,21.0
2018-12-31 21:30:00,1874.002081,2018.0,12.0,0.0,31.0,21.0,30.0,2018-12-31 21:30:00-06:00,61.27,46.40,2019-01-01 03:30:00,21.0
2018-12-31 21:45:00,1988.168511,2018.0,12.0,0.0,31.0,21.0,45.0,2018-12-31 21:45:00-06:00,65.60,44.60,2019-01-01 03:45:00,21.0
2018-12-31 22:00:00,2022.795943,2018.0,12.0,0.0,31.0,22.0,0.0,2018-12-31 22:00:00-06:00,65.60,44.60,2019-01-01 04:00:00,22.0
2018-12-31 22:15:00,1986.981872,2018.0,12.0,0.0,31.0,22.0,15.0,2018-12-31 22:15:00-06:00,65.60,44.60,2019-01-01 04:15:00,22.0
2018-12-31 22:30:00,1921.142106,2018.0,12.0,0.0,31.0,22.0,30.0,2018-12-31 22:30:00-06:00,65.60,44.60,2019-01-01 04:30:00,22.0
2018-12-31 22:45:00,1956.130266,2018.0,12.0,0.0,31.0,22.0,45.0,2018-12-31 22:45:00-06:00,65.60,44.60,2019-01-01 04:45:00,22.0
2018-12-31 23:00:00,2039.546964,2018.0,12.0,0.0,31.0,23.0,0.0,2018-12-31 23:00:00-06:00,65.60,44.60,2019-01-01 05:00:00,23.0
2018-12-31 23:15:00,2100.701840,2018.0,12.0,0.0,31.0,23.0,15.0,2018-12-31 23:15:00-06:00,70.27,42.80,2019-01-01 05:15:00,23.0


In [17]:
base_df 

,demand_kW,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC,hour
datetime_local,,,,,,,,,,,,
2018-12-31 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00,18.0
2018-12-31 18:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00,18.0
2018-12-31 18:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00,18.0
2018-12-31 18:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00,18.0
2018-12-31 18:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00,18.0
2018-12-31 18:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00,18.0
2018-12-31 18:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00,18.0
2018-12-31 18:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00,18.0
2018-12-31 18:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00,18.0


Dummy drops things that are NA, maybe sth to look at

In [4]:
dummy_df = base_df.dropna(subset=['demand_kW', 'Temperature', 'Relative Humidity']) 
dummy_df = dummy_df.reset_index()
dummy_df = dummy_df.drop(['hour'], axis=1)
dummy_df = dummy_df.drop(['datetime'], axis=1)
dummy_df = dummy_df.drop(['datetime_local'], axis=1)
dummy_df = dummy_df.drop(['datetime_UTC'], axis=1)
dummy_df

,demand_kW,year,month,weekday,day,minute,Relative Humidity,Temperature
0,2064.101392,2018.0,12.0,0.0,31.0,15.0,61.27,46.40
1,1874.002081,2018.0,12.0,0.0,31.0,30.0,61.27,46.40
2,1988.168511,2018.0,12.0,0.0,31.0,45.0,65.60,44.60
3,2022.795943,2018.0,12.0,0.0,31.0,0.0,65.60,44.60
4,1986.981872,2018.0,12.0,0.0,31.0,15.0,65.60,44.60
5,1921.142106,2018.0,12.0,0.0,31.0,30.0,65.60,44.60
6,1956.130266,2018.0,12.0,0.0,31.0,45.0,65.60,44.60
7,2039.546964,2018.0,12.0,0.0,31.0,0.0,65.60,44.60
8,2100.701840,2018.0,12.0,0.0,31.0,15.0,70.27,42.80
9,1901.260583,2018.0,12.0,0.0,31.0,30.0,75.32,41.00


In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error

# Train and Testset
Things to look at:
1. how to divide test/train/holdout set

In [6]:
train, test = train_test_split(dummy_df)
X_train = train.copy().drop(['demand_kW'], axis=1)
Y_train = train['demand_kW']
X_test = test.copy().drop(['demand_kW'], axis=1)
Y_test = test['demand_kW']

# Model Testing


## SVM

In [12]:
svm = SVR()
svm.fit(X_train, Y_train)
acc = svm.predict(X_test)
result = r2_score(list(Y_test), acc)
result

C:\Users\sonja\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


-6.817829768035466e-05